In [3]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import preprocessing
# from src2.model import *
pd.options.display.max_columns = 100


train = pd.read_csv('../data/train_dataset.csv')
test = pd.read_csv('../data/test_dataset.csv')
target = train['信用分']
data = pd.concat([train,test],axis=0,ignore_index=True)
data0 = data.copy()
data['信用分'] = data['信用分'].fillna(-1)



def feature_count(data, features):
    if len(set(data[features])) == len(data[features]):
        print('equal feature')
        return data
    new_feature = 'count'
    new_feature += '_' + features
    temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
    data = data.merge(temp, 'left', on=features)
    return data

def endnumber(x):
    x_2_str = str(x)
    res = x_2_str.split(".")
    return int(res[-1])

def freq_coding(x, countdict):
    if (countdict[x] > 0) and (countdict[x] <= 50):
        return 0
    elif (countdict[x] > 50) and (countdict[x] <= 300):
        return 1
    else:
        return 2

def omit_value(x):
    if x <= 500:
        return 0
    elif (x>500) and (x<1000):
        return 1
    else:
        return 2

def standard(x):
    return preprocessing.StandardScaler().fit_transform(x.values.reshape(-1,1))

class Feature:
    def __init__(self, data):
        self.name = 'feature1'
        self.data = data
#         self.predictors = predictors
    
    def single_data_distribution(self):
    # identity 3
        self.data.loc[self.data['用户年龄'] == 0, '用户年龄'] = np.nan
        #
        self.data['month'] = self.data['用户网龄（月）'] // 12
        #
        self.data['year'] = self.data['用户网龄（月）'] % 12
        
    # business 2
        self.data.loc[self.data['用户话费敏感度'] == 0, '用户话费敏感度'] = 1 # self.data['用户话费敏感度'].median()
        # 充值路径
        self.data['charge_type'] = 0
        self.data.loc[(self.data['缴费用户最近一次缴费金额（元）'] % 10 == 0) & (self.data['缴费用户最近一次缴费金额（元）'] != 0), 'charge_type'] = 1

    def single_feature_count(self):
    # business 4
        feature_count_list = ['缴费用户最近一次缴费金额（元）', '用户近6个月平均消费值（元）', '用户当月账户余额（元）', '用户账单当月总费用（元）']
        for ff in feature_count_list:
            self.data = feature_count(self.data, ff)
            

    def composite_business(self):
    # business 4
        # 各种比例 3
        self.data['user_cost_delt'] = self.data['用户近6个月平均消费值（元）'] - self.data['用户账单当月总费用（元）']
        # 过去5个月和当月的消费情况 2
        self.data['last_five_fee'] = self.data['用户近6个月平均消费值（元）']*6 - self.data['用户账单当月总费用（元）']
        self.data['now-five'] = self.data['用户账单当月总费用（元）'] - self.data['last_five_fee']/5
        
        # app 1
        self.data['buy_rate'] = self.data['当月物流快递类应用使用次数'] / (self.data['当月网购类应用使用次数'] + 1)

    
    def drop_columns_na(self):
        self.data.drop(columns=['是否大学生客户','是否黑名单客户','是否经常逛商场的人'], inplace=True)
        self.data.dropna(inplace=True)
        self.data.reset_index(drop=True, inplace=True)
        
    def save_data(self):
        print("输出数据的尺寸:", self.data.shape)
        self.data.to_csv("../data/all_data_fea3.csv", index=False, encoding='utf-8')

/home/wrj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [4]:
case = Feature(data)
case.single_data_distribution()
case.single_feature_count()
case.composite_business()
case.drop_columns_na()
case.save_data()


输出数据的尺寸: (99462, 38)
